| Option                     | Ye kya hai                       | Kab use kare                            | Pros                        | Cons                            |
| -------------------------- | -------------------------------- | --------------------------------------- | --------------------------- | ------------------------------- |
| **Console**                | Google Cloud ka **UI/Dashboard** | First-time demo, learning, quick tuning | No code, visual monitoring  | Automation nahi, slow for scale |
| **Google Gen AI SDK**      | **Gemini-first Python SDK**      | Gemini apps, fine-tuning, inference     | Simple, Gemini-optimized    | Gemini tak focused              |
| **Vertex AI SDK (Python)** | **Full Vertex AI Python SDK**    | Enterprise ML, open-source models       | MLOps, endpoints, pipelines | Thoda heavy / verbose           |
| **REST API**               | Raw **HTTP calls**               | Any language / custom backend           | Language-agnostic, flexible | Manual auth, verbose            |
| **Colab Enterprise**       | Managed **enterprise notebooks** | Team training, security, governance     | IAM, VPC, compliance        | Paid, overkill for solo dev     |


## Install the google sdk

In [1]:
!pip install --upgrade google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 724.2/724.2 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.60.0
    Uninstalling google-genai-1.60.0:
      Successfully uninstalled google-genai-1.60.0


In [4]:
PROJECT_ID = "gen-lang-client-0486212375"
LOCATION = "us-central1"

# If you want evaluation output:
# OUTPUT_GCS_URI = "gs://gemini-sft-custom/eval-results/"

## Import the required packages

| Import Statement                                               | Why It Is Used                                                                                                                       |
| -------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------ |
| `import time`                                                  | Used to pause execution while polling the fine-tuning job status (e.g., `sleep(60)` to avoid frequent API calls).                    |
| `from google import genai`                                     | Core Google GenAI SDK used to interact with Gemini models, create fine-tuning jobs, list jobs, fetch job details, and run inference. |
| `from google.genai.types import HttpOptions`                   | Used to control API behavior such as selecting the API version (`v1beta1` for fine-tuning, `v1` for inference and job retrieval).    |
| `from google.genai.types import CreateTuningJobConfig`         | Used to define configuration for the supervised fine-tuning job, including the tuned model name and optional evaluation settings.    |
| `from google.genai.types import TuningDataset`                 | Used to reference the training dataset stored in Google Cloud Storage (JSONL format) without loading data locally.                   |
| `from google.colab import auth`                                | Used to authenticate the Colab environment with the user’s Google Cloud account to access Vertex AI resources.                       |
| `from google.cloud import aiplatform`                          | Vertex AI SDK used for model registry operations such as deleting the fine-tuned model during cleanup.                               |
| `from google.genai.types import EvaluationConfig` *(optional)* | Used only if automated evaluation is enabled after fine-tuning to define evaluation behavior and metrics.                            |
| `from google.genai.types import OutputConfig` *(optional)*     | Specifies where evaluation outputs should be stored in Google Cloud Storage.                                                         |
| `from google.genai.types import GcsDestination` *(optional)*   | Defines the Cloud Storage destination for evaluation results.                                                                        |
| `from google.genai.types import Metric` *(optional)*           | Used to define evaluation metrics such as fluency or accuracy during post-training evaluation.                                       |


## Authenticate from the colab

In [6]:
from google.colab import auth
auth.authenticate_user()

## Initlized the client

In [ ]:
import time
from google import genai
from google.genai.types import HttpOptions, CreateTuningJobConfig, TuningDataset
from google.colab import auth
from google.cloud import aiplatform

In [7]:
client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(api_version="v1beta1")
)

## Make the request to the model

In [8]:
resp = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="What is a color of the water?",
)

In [9]:
print(resp.text)

OK



Init the training dataset

In [8]:
from google.genai.types import TuningDataset

training_dataset = TuningDataset(
    gcs_uri="gs://gemini-sft-custom/data.jsonl",
)


## Create a finetuning job

In [9]:
from google.genai.types import CreateTuningJobConfig

tuning_job = client.tunings.tune(
    base_model="gemini-2.5-flash",
    training_dataset=training_dataset,
    config=CreateTuningJobConfig(
        tuned_model_display_name="example-sft-job",
    ),
)

/tmp/ipython-input-1133439047.py:3: ExperimentalWarning: The SDK's tuning implementation is experimental, and may change in future versions.
  tuning_job = client.tunings.tune(


In [14]:
print("Tuning Job:", tuning_job.name)

NameError: name 'tuning_job' is not defined

In [16]:
jobs = client.tunings.list()
for j in jobs:
    print(j.name)

projects/313902242452/locations/us-central1/tuningJobs/6774416233392504832
projects/313902242452/locations/us-central1/tuningJobs/5422747006948868096


In [17]:
tuning_job_name = jobs[0].name  # latest job
tuning_job = client.tunings.get(name=tuning_job_name)

print("STATE:", tuning_job.state)
print("MODEL:", tuning_job.tuned_model.model)
print("ENDPOINT:", tuning_job.tuned_model.endpoint)

STATE: JobState.JOB_STATE_SUCCEEDED
MODEL: projects/313902242452/locations/us-central1/models/2360465647369977856@1
ENDPOINT: projects/313902242452/locations/us-central1/endpoints/7655379395204349952


In [18]:
response = client.models.generate_content(
    model=tuning_job.tuned_model.endpoint,
    contents="Why is the sky blue?"
)
print(response.text)

The sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's a breakdown:

1.  **Sunlight is White Light:** Sunlight, when it reaches Earth, appears white, but it's actually composed of all the colors of the rainbow (red, orange, yellow, green, blue, indigo, violet). Each color has a different wavelength, with violet/blue having the shortest wavelengths and red/orange having the longest wavelengths.

2.  **Earth's Atmosphere:** Our atmosphere is made up of tiny gas molecules, primarily nitrogen (about 78%) and oxygen (about 21%). These molecules are much smaller than the wavelengths of visible light.

3.  **Rayleigh Scattering:** When sunlight enters the atmosphere, it interacts with these tiny gas molecules. Because these molecules are much smaller than the wavelengths of light, they scatter shorter wavelengths (blue and violet) much more effectively than longer wavelengths (red and orange). This specific type of scattering is called Rayleigh scattering.

4.  **Why 

## Wait till training completes (recommended)

In [11]:
import time

running_states = {"JOB_STATE_PENDING", "JOB_STATE_RUNNING"}

while tuning_job.state in running_states:
    print("State:", tuning_job.state)
    tuning_job = client.tunings.get(name=tuning_job.name)
    time.sleep(60)

print("Final State:", tuning_job.state)
print("MODEL:", tuning_job.tuned_model.model)
print("ENDPOINT:", tuning_job.tuned_model.endpoint)


NameError: name 'tuning_job' is not defined

In [12]:
client_v1 = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(api_version="v1")
)


## Inference with tuned model (FINAL TEST)

In [13]:

response = client_v1.models.generate_content(
    model=tuning_job.tuned_model.endpoint,
    contents="Why is the sky blue?",
)
print(response.text)


NameError: name 'tuning_job' is not defined

## model clean-up

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

MODEL_ID = tuning_job.tuned_model.model  # <-- use the actual model resource name
model = aiplatform.Model(MODEL_ID)
model.delete()
print("Deleted model:", MODEL_ID)